In [92]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb

# Get Data

In [291]:
def import_data():
    dfx = pd.read_csv('../data/x_train.csv').set_index('ID')
    dfy = pd.read_csv('../data/y_train.csv').set_index('ID')
    dfx_test = pd.read_csv('../data/x_test.csv').set_index('ID')
    return dfx, dfy, dfx_test

dfx, dfy, dfx_test = import_data()
dfx.head()

,neuron_id,timestamp_0,timestamp_1,timestamp_2,timestamp_3,timestamp_4,timestamp_5,timestamp_6,timestamp_7,timestamp_8,...,timestamp_40,timestamp_41,timestamp_42,timestamp_43,timestamp_44,timestamp_45,timestamp_46,timestamp_47,timestamp_48,timestamp_49
ID,,,,,,,,,,,,,,,,,,,,,
0,53,0.166262,0.170520,0.176371,0.197565,0.212814,0.957631,1.177679,3.207268,3.213602,...,14.700766,15.856531,15.906414,15.910547,16.079702,16.516175,16.988851,16.996101,17.019988,17.056403
1,7229,0.053270,0.133652,0.161603,0.530531,0.692537,0.730291,0.784066,1.023311,1.059203,...,10.420048,11.577627,18.070029,18.825891,19.248378,19.539560,19.723312,19.978294,20.052356,20.123709
2,7770,0.600593,1.127467,1.289813,2.598660,3.826711,4.851020,5.025722,5.262254,5.282915,...,24.635121,25.967167,26.270768,28.028523,29.584844,31.183257,31.218086,31.285777,33.160938,33.360947
3,7002,0.136037,0.655734,0.992514,1.490964,2.246825,2.855319,3.208054,3.325092,4.202108,...,13.473805,13.688518,14.028203,14.178543,14.783635,15.151632,16.003356,16.282885,17.249543,17.259610
4,7678,0.200614,0.257171,0.288137,0.314078,0.486903,0.499591,0.534961,0.623986,0.639313,...,11.298341,12.214740,13.072630,13.952573,14.013957,14.476091,14.724149,16.883795,17.052340,19.289922


In [292]:
def differencing(df):
    columns = df.columns
    new_df = np.concatenate((df.iloc[:,:2], df.iloc[:,2:].values - df.iloc[:,1:-1].values), axis=1)
    new_df = pd.DataFrame(new_df, columns=columns)
    new_df = new_df.drop(['timestamp_0'], axis=1)
    return new_df

dfx = differencing(dfx)
dfx.head()

,neuron_id,timestamp_1,timestamp_2,timestamp_3,timestamp_4,timestamp_5,timestamp_6,timestamp_7,timestamp_8,timestamp_9,...,timestamp_40,timestamp_41,timestamp_42,timestamp_43,timestamp_44,timestamp_45,timestamp_46,timestamp_47,timestamp_48,timestamp_49
0,53.0,0.004258,0.005851,0.021194,0.015249,0.744817,0.220048,2.029589,0.006334,1.138758,...,0.023318,1.155765,0.049883,0.004133,0.169155,0.436473,0.472676,0.007250,0.023887,0.036415
1,7229.0,0.080382,0.027950,0.368929,0.162005,0.037755,0.053775,0.239245,0.035892,0.012870,...,0.027535,1.157579,6.492402,0.755862,0.422487,0.291182,0.183752,0.254982,0.074063,0.071353
2,7770.0,0.526874,0.162346,1.308847,1.228051,1.024309,0.174702,0.236532,0.020661,1.324287,...,0.051216,1.332046,0.303601,1.757755,1.556321,1.598413,0.034828,0.067691,1.875161,0.200009
3,7002.0,0.519697,0.336779,0.498451,0.755861,0.608494,0.352735,0.117038,0.877016,0.988270,...,0.008387,0.214713,0.339685,0.150340,0.605092,0.367997,0.851724,0.279529,0.966659,0.010066
4,7678.0,0.056557,0.030966,0.025940,0.172825,0.012688,0.035370,0.089025,0.015327,0.252605,...,0.185276,0.916400,0.857890,0.879943,0.061384,0.462134,0.248058,2.159646,0.168545,2.237582


In [293]:
def sommes(df_diff, step="train"):
    """ Add 5 "sommes" (=quantiles) taken from the original spike trains to represent spike train length."""
    if step == "train":
        df = pd.read_csv('../data/x_train.csv').set_index('ID')
    else:
        df = pd.read_csv('../data/x_test.csv').set_index('ID')
        
    #somme_tokeep = [10, 20, 30, 40, 50]
    somme_tokeep = [30, 50]
    columns = ["somme_" + str(s) for s in somme_tokeep]
    somme = df.iloc[:,np.array(somme_tokeep)]
    somme.columns = columns
    new_df = pd.concat((df_diff, somme), axis=1)
    return new_df

dfx = sommes(dfx, "train")
dfx.head()

,neuron_id,timestamp_1,timestamp_2,timestamp_3,timestamp_4,timestamp_5,timestamp_6,timestamp_7,timestamp_8,timestamp_9,...,timestamp_42,timestamp_43,timestamp_44,timestamp_45,timestamp_46,timestamp_47,timestamp_48,timestamp_49,somme_30,somme_50
0,53.0,0.004258,0.005851,0.021194,0.015249,0.744817,0.220048,2.029589,0.006334,1.138758,...,0.049883,0.004133,0.169155,0.436473,0.472676,0.007250,0.023887,0.036415,8.983281,17.056403
1,7229.0,0.080382,0.027950,0.368929,0.162005,0.037755,0.053775,0.239245,0.035892,0.012870,...,6.492402,0.755862,0.422487,0.291182,0.183752,0.254982,0.074063,0.071353,6.322775,20.123709
2,7770.0,0.526874,0.162346,1.308847,1.228051,1.024309,0.174702,0.236532,0.020661,1.324287,...,0.303601,1.757755,1.556321,1.598413,0.034828,0.067691,1.875161,0.200009,17.311853,33.360947
3,7002.0,0.519697,0.336779,0.498451,0.755861,0.608494,0.352735,0.117038,0.877016,0.988270,...,0.339685,0.150340,0.605092,0.367997,0.851724,0.279529,0.966659,0.010066,11.257401,17.259610
4,7678.0,0.056557,0.030966,0.025940,0.172825,0.012688,0.035370,0.089025,0.015327,0.252605,...,0.857890,0.879943,0.061384,0.462134,0.248058,2.159646,0.168545,2.237582,7.997795,19.289922


In [294]:
def manual_features(df):
    df_diff = df.iloc[:,1:50]
    df["mean"] = df_diff.mean(axis=1)
    df["min"] = df_diff.min(axis=1)    
    df["max"] = df_diff.max(axis=1)
    quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    for q in quantiles:
        df["quant_" + str(q)] = df_diff.quantile(q, axis=1)
    return df

dfx = manual_features(dfx)

In [295]:
### VERY IMPORTANT STEP: Delete neuron_id, and timesteps columns  ###
dfx = dfx.iloc[:,50:]
print(dfx.head())

X_train, X_val, y_train, y_val = train_test_split(dfx, dfy, test_size=0.2, random_state=42)
y_train = np.squeeze(y_train) #add dimension to silence warnings from sklearn...
y_val = np.squeeze(y_val) #add dimension to silence warnings from sklearn...

    somme_30   somme_50      mean       min       max  quant_0.1  quant_0.2  \
0   8.983281  17.056403  0.344697  0.004133  2.838052   0.006246   0.011531   
1   6.322775  20.123709  0.409601  0.008825  6.492402   0.022980   0.032715   
2  17.311853  33.360947  0.668579  0.014021  3.185310   0.047949   0.105754   
3  11.257401  17.259610  0.349461  0.004493  1.077179   0.010515   0.046033   
4   7.997795  19.289922  0.389578  0.012688  2.653620   0.025003   0.058350   

   quant_0.3  quant_0.4  quant_0.5  quant_0.6  quant_0.7  quant_0.8  quant_0.9  
0   0.020297   0.027357   0.064258   0.093709   0.223973   0.409775   1.142159  
1   0.072437   0.098431   0.188263   0.243122   0.319462   0.406532   0.764169  
2   0.196590   0.296318   0.342279   0.582877   1.001320   1.291991   1.564740  
3   0.139411   0.203916   0.309909   0.350125   0.477036   0.606453   0.783913  
4   0.093700   0.151082   0.185276   0.251696   0.295910   0.478790   0.908249  


# Train model

* **Random Forest:**

In [296]:
def train_rf(X_train, y_train):
    rf = RandomForestClassifier(random_state=42, class_weight='balanced')

    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 800, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]

    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}

    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 6, cv = 3, verbose=0, random_state=42, n_jobs = -1, scoring='roc_auc')
    rf_random.fit(X_train, y_train)

    best_random = rf_random.best_estimator_
    print("RF best CV ROC_AUC score: ", rf_random.best_score_)
    return best_random

In [297]:
best_random = train_rf(X_train, y_train)
y_pred_val = best_random.predict(X_val)
print("ROC_AUC: ", metrics.roc_auc_score(y_val, y_pred_val))
print("CKS: ", metrics.cohen_kappa_score(y_val, y_pred_val))

RF best CV ROC_AUC score:  0.7555392495395948
ROC_AUC:  0.620588643453182
CKS:  0.28226451388072393


* **XGBoost:**

In [298]:
def train_xgb(X_train, y_train):
    # Class balancing trick
    scale_pos_weight = np.sum(y_train == 0)/ float(np.sum(y_train == 1))

    model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

    params_XGB = {
            'min_child_weight': [0.1, 1, 3],
            'subsample': [0.6, 0.8, 1],
            'colsample_bytree': [0.6],
            'max_depth': [3, 6, 10],
            'n_estimators': [200],
            'learning_rate': [0.05, 0.1],
            'gamma': [0.5, 1],
            'scale_pos_weight': [scale_pos_weight]
            }

    search = RandomizedSearchCV(model, param_distributions=params_XGB, n_iter=1, scoring='roc_auc', n_jobs=4, 
                                verbose=0, random_state=1001 )

    search.fit(X_train, y_train)
    #search.fit(select_X_train, y_train)
    best_xgb = search.best_estimator_
    print("XGB best CV ROC_AUC score: ", search.best_score_)
    return best_xgb

In [299]:
best_xgb = train_xgb(X_train, y_train)
y_pred_val = best_xgb.predict(X_val)
print("ROC_AUC: ", metrics.roc_auc_score(y_val, y_pred_val))
print("CKS: ", metrics.cohen_kappa_score(y_val, y_pred_val))

XGB best CV ROC_AUC score:  0.7493338230780341
ROC_AUC:  0.689900565353542
CKS:  0.2957644995300206


# Train groupby model

In [300]:
def groupby_neurons(df):
    # Get rid of the 50 spike train timesteps
    df_features = dfx.iloc[:, 50:]
    df_features["neuron_id"] = dfx["neuron_id"]
    
    # Group the samples by neuron_id
    grouped = df_features.groupby("neuron_id").mean()
    
    # Merge the features_df and the grouped_df on neuron_id column, and keep the original sample order
    final = df_features.reset_index().merge(grouped, on='neuron_id', sort=False).sort_values('index').set_index('index')
    return final

In [301]:
dfx, dfy, dfx_test = import_data()
dfx = differencing(dfx)
dfx = sommes(dfx)
dfx = manual_features(dfx)
dfx_nid = groupby_neurons(dfx)
dfx_nid.head()

,somme_30_x,somme_50_x,mean_x,min_x,max_x,quant_0.1_x,quant_0.2_x,quant_0.3_x,quant_0.4_x,quant_0.5_x,...,max_y,quant_0.1_y,quant_0.2_y,quant_0.3_y,quant_0.4_y,quant_0.5_y,quant_0.6_y,quant_0.7_y,quant_0.8_y,quant_0.9_y
index,,,,,,,,,,,,,,,,,,,,,
0,8.983281,17.056403,0.344697,0.004133,2.838052,0.006246,0.011531,0.020297,0.027357,0.064258,...,5.829577,0.006480,0.010037,0.017278,0.038152,0.082100,0.186866,0.410137,0.995403,2.002561
1,6.322775,20.123709,0.409601,0.008825,6.492402,0.022980,0.032715,0.072437,0.098431,0.188263,...,3.449927,0.023353,0.043877,0.072804,0.115968,0.187514,0.288325,0.460189,0.771798,1.262314
2,17.311853,33.360947,0.668579,0.014021,3.185310,0.047949,0.105754,0.196590,0.296318,0.342279,...,2.604903,0.056812,0.098609,0.149527,0.213957,0.297770,0.409597,0.562768,0.784305,1.165241
3,11.257401,17.259610,0.349461,0.004493,1.077179,0.010515,0.046033,0.139411,0.203916,0.309909,...,2.161103,0.021653,0.050344,0.091367,0.146018,0.209738,0.301930,0.423963,0.607302,0.929014
4,7.997795,19.289922,0.389578,0.012688,2.653620,0.025003,0.058350,0.093700,0.151082,0.185276,...,2.959074,0.019778,0.047907,0.096047,0.163873,0.251328,0.372587,0.540685,0.785203,1.220172


In [302]:
def custom_split(dfx, dfy):
    """ Custom train_test_split function to keep the model from seeing the neuron_ids in X_val samples. """
    # Sample N random neuron_ids
    val_set = np.random.choice(pd.unique(dfx["neuron_id"]), 60, replace=False)
    val_idx = dfx['neuron_id'].isin(val_set)
    
    # Get X_train y_train, X_val y_val
    X_train = dfx[~val_idx].drop(["neuron_id"], axis=1)
    X_val = dfx[val_idx].drop(["neuron_id"], axis=1)
    y_train = dfy[~val_idx]
    y_val = dfy[val_idx]
    
    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = custom_split(dfx_nid, dfy)
y_train = np.squeeze(y_train) #Delete useless dimension to silence warnings from sklearn...
y_val = np.squeeze(y_val) #Delete useless dimension to silence warnings from sklearn...

In [303]:
best_xgb_nid = train_xgb(X_train, y_train)
y_pred_val = best_xgb_nid.predict(X_val)
print("ROC_AUC: ", metrics.roc_auc_score(y_val, y_pred_val))
print("CKS: ", metrics.cohen_kappa_score(y_val, y_pred_val))

XGB best CV ROC_AUC score:  0.8997682216351574
ROC_AUC:  0.7290277777777777
CKS:  0.3231115900852356


In [304]:
best_random_nid = train_rf(X_train, y_train)
y_pred_val = best_random_nid.predict(X_val)
print("ROC_AUC: ", metrics.roc_auc_score(y_val, y_pred_val))
print("CKS: ", metrics.cohen_kappa_score(y_val, y_pred_val))

RF best CV ROC_AUC score:  0.9020943280943498
ROC_AUC:  0.6205324074074073
CKS:  0.21763845350052258
